### Problem 8 hetmanów 


[Wikipedia](https://pl.wikipedia.org/wiki/Problem_o%C5%9Bmiu_hetman%C3%B3w)  
<p>Problem ośmiu hetmanów – hetman (zwany też królową) jest figurą szachową, która bije figury znajdujące się w tej samej kolumnie, wierszu lub przekątnej, co on sam.</p>
<p>W jaki sposób rozstawić osiem hetmanów na tradycyjnej szachownicy 8x8 tak, aby wzajemnie się nie atakowały? Ile jest możliwych rozstawień?</p>
<p>Przez rozstawienie podstawowe bądź rozwiązanie podstawowe należy rozumieć rozwiązanie z dokładnością do izomorfizmu, tzn. z uwzględnieniem wszystkich pokrewnych pozycji wynikających z odbić zwierciadlanych i obrotów.</p>

#### Zakres ruchów hetmana
![Hetman](queen.png "Hetman i jego ruchy")


**Genotyp** to sposób kodowania części problemu, dzięki czemu można nimi manipulować za pomocą algorytmu genetycznego.  
Przykład: potencjalne genotypy dla tego problemu (chodzi o 8 pozycji, w których znajdują się hetmany):  
  * 64 bity, po jednym dla każdego z 64 kwadratów na planszy 
  * 48 bitów, po 6 dla każdej z lokalizacjihetmana, ponieważ możemy policzyć do 64 za pomocą 6 bitów  
  * 8 liczb całkowitych z zakresu 0..63 lub 1..64  
  * 16 liczb całkowitych reprezentujących położenie wiersza i kolumny każdego hetmana  
  
**Fenotyp** mówi nam, jak odkodowane geny są wykorzystywane do rozwiązania problemu.  
W każdym z powyższych przykładów fenotyp to lokalizacje 8 hetmanów na planszy.  

**Funkcja sprawności (ang. fitness)** ocenia następnie fenotyp w kontekście rozwiązania problemu w celu obliczenia  wartości funkcji sprawności.

Na początek musimy zdefiniować **genotyp**. Użyjemy dwóch genów dla pozycji każdej królowej - po jednym dla każdego rzędu i kolumny.  
Szachownica ma tę samą liczbę rzędów co kolumn (8), więc użyjemy cyfr 0-7.

In [1]:
# Do roboty
# Import niezbędnych modułów
import datetime
import random
import statistics
import sys
import time

In [2]:
# klasa przechowująca informacje o genach i ich wartości funkcji 'fitness'
class Chromosome:
    def __init__(self, genes, fitness):
        self.Genes = genes
        self.Fitness = fitness

In [3]:
# klasa związana z szachownicą
class Board:
    # konstruktor inicjujący szachownicę losowym położeniem hetmanów
    def __init__(self, genes, size):
        board = [['.'] * size for _ in range(size)]
        # konwersja str->int
        genes = [int(i) for i in genes]
        for index in range(0, len(genes), 2):
            row = genes[index]
            column = genes[index + 1]
            board[column][row] = 'Q'
        self._board = board

    # rysowanie szachownicy z hetmanami
    def print(self):
        # 0,0 prints in bottom left corner
        for i in reversed(range(len(self._board))):
            print(' '.join(self._board[i]))
            
    # zwracamy konkretną pozycję hetmana
    def get(self, row, column):
        return self._board[column][row]


In [4]:
"""
Generujemy 16 liczb z zakresu <0, 7>, czyli 8 razy po dwie współrzędne hetmana na szachownicy.
Pierwsza współrzędna to numer kolumny (od 0 do 7), a druga to numer wiersza (również od 0 do 7)
I tak 8 razy ...
(0, 0) to lewy dolny róg, a (7, 7) to prawy górny róg.
"""
# 
genes1 = [random.randint(0, 7) for i in range(16)]
print("Losowe współrzędne:", genes1)
board1 = Board(genes1, 8)
board1.print()

Losowe współrzędne: [2, 4, 6, 3, 1, 6, 1, 5, 6, 5, 1, 5, 1, 5, 3, 4]
. . . . . . . .
. Q . . . . . .
. Q . . . . Q .
. . Q Q . . . .
. . . . . . Q .
. . . . . . . .
. . . . . . . .
. . . . . . . .


### Fitness

Jeden ze sposobów rozwiązania jest następujący: 
  * zliczamy liczbę wierszy, kolumn i przekątnych (z góry po skosie na dół, czyli northEast oraz z dołu po skosie do góry, czyli southEast), które mają królowe, aby określić, ile jest atakowanych.
  * wartość sprawności będzie sumą tych czterech liczb, odjętą od wartości maksymalnej (8 + 8 + 8 + 8 lub 32). 
  * to znaczy optymalna wartość będzie wynosić zero, a wyższe wartości będą gorsze.

In [5]:
# najpierw zdefiniujmy klasę Fitness
class Fitness:
    def __init__(self, total):
        self.Total = total

    def __gt__(self, other):
        return self.Total < other.Total
    
    def __ge__(self, other):
        return self.Total <= other.Total

    def __str__(self):
        return "{}".format(self.Total)

In [6]:
# obliczanie wartości fitness
def get_fitness(genes, size):
    board = Board(genes, size)
    rowsWithQueens = set()
    colsWithQueens = set()
    northEastDiagonalsWithQueens = set()
    southEastDiagonalsWithQueens = set()
    for row in range(size):
        for col in range(size):
            if board.get(row, col) == 'Q':
                rowsWithQueens.add(row)
                colsWithQueens.add(col)
                northEastDiagonalsWithQueens.add(row + col)
                southEastDiagonalsWithQueens.add(size - 1 - row + col)
    total = size - len(rowsWithQueens) \
            + size - len(colsWithQueens) \
            + size - len(northEastDiagonalsWithQueens) \
            + size - len(southEastDiagonalsWithQueens)
    return Fitness(total)

### Funkcje pomocnicze

In [7]:
# wizualizacja lokalizacji hetmanów
def display(candidate, startTime, size):
    timeDiff = datetime.datetime.now() - startTime
    board = Board(candidate.Genes, size)
    board.print()
    print("Geny: {},\t fitness - {}, \tczas - {}".format(
        ' '.join(map(str, candidate.Genes)),
        candidate.Fitness,
        timeDiff))

In [8]:
# szukanie najlepszego osobnika
def get_best(get_fitness, targetLen, optimalFitness, geneSet, display):
    random.seed()
    bestParent = _generate_parent(targetLen, geneSet, get_fitness)
    display(bestParent)
    if bestParent.Fitness > optimalFitness:
        return bestParent
    
    # uruchom tylko na kilka sekund
    czas_dzialania = 3
    t_end = time.time() + czas_dzialania

    while time.time() < t_end:
        child = _mutate(bestParent, geneSet, get_fitness)
        if bestParent.Fitness > child.Fitness:
            continue
        display(child)
            
        if child.Fitness > optimalFitness:
            return child
        bestParent = child
        
        

In [9]:
# generowanie rodzica
def _generate_parent(length, geneSet, get_fitness):
    genes = []
    while len(genes) < length:
        sampleSize = min(length - len(genes), len(geneSet))
        genes.extend(random.sample(geneSet, sampleSize))
    # konwersja int->str
    genes = [str(i) for i in genes]

    genes = ''.join(genes)
    fitness = get_fitness(genes)
    return Chromosome(genes, fitness)

In [10]:
# mutacja
def _mutate(parent, geneSet, get_fitness):
    index = random.randrange(0, len(parent.Genes))
    childGenes = list(parent.Genes)
    newGene, alternate = random.sample(geneSet, 2)
    childGenes[index] = str(alternate) if str(newGene) == childGenes[index] else str(newGene)
    genes = ''.join(childGenes)
    fitness = get_fitness(genes)
    return Chromosome(genes, fitness)

In [11]:
def start(size=8):
        geneset = [i for i in range(size)]
        startTime = datetime.datetime.now()

        def fnDisplay(candidate):
            display(candidate, startTime, size)

        def fnGetFitness(genes):
            return get_fitness(genes, size)

        optimalFitness = Fitness(0)
        best = get_best(fnGetFitness, 2 * size, optimalFitness,
                                geneset, fnDisplay)
        
start(8)

Q . . . . . . .
. . . Q . . . .
. . . . . . . .
. Q . . . . Q .
. . . . . . . .
. . . . . Q . Q
. . . Q . . . .
. . . . . Q . .
Geny: 1 4 3 6 5 0 7 2 3 1 5 2 0 7 6 4,	 fitness - 10, 	czas - 0:00:00.002022
. . . . . . . .
Q . . Q . . . .
. . . . . . . .
. Q . . . . Q .
. . . . . . . .
. . . . . Q . Q
. . . Q . . . .
. . . . . Q . .
Geny: 1 4 3 6 5 0 7 2 3 1 5 2 0 6 6 4,	 fitness - 10, 	czas - 0:00:00.005385
. . . . . . . .
Q . . Q . . . .
. . . . . . . .
. . Q . . . Q .
. . . . . . . .
. . . . . Q . Q
. . . Q . . . .
. . . . . Q . .
Geny: 2 4 3 6 5 0 7 2 3 1 5 2 0 6 6 4,	 fitness - 9, 	czas - 0:00:00.007757
. . . . . . . .
Q . . . Q . . .
. . . . . . . .
. . Q . . . Q .
. . . . . . . .
. . . . . Q . Q
. . . Q . . . .
. . . . . Q . .
Geny: 2 4 4 6 5 0 7 2 3 1 5 2 0 6 6 4,	 fitness - 9, 	czas - 0:00:00.010356
. . . . . . . .
Q . . . Q . . .
. . Q . . . . .
. . . . . . Q .
. . . . . . . .
. . . . . Q . Q
. . . Q . . . .
. . . . . Q . .
Geny: 2 5 4 6 5 0 7 2 3 1 5 2 0 6 6 4,	 fitness - 7, 	

. Q . . . . . .
. . . . . . . .
. . . . . Q . Q
. . . Q . . . .
. . . . . . Q .
Q . . . . . . .
. . Q . . . . .
. . . . Q . . .
Geny: 2 1 6 3 4 0 5 5 7 5 3 4 0 2 1 7,	 fitness - 1, 	czas - 0:00:00.214032
. Q . . . . . .
. . . . . . . .
. . . . . . . Q
. . . Q . . . .
. . . . . . Q .
Q . . . . . . .
. . Q . . . . .
. . . . Q Q . .
Geny: 2 1 6 3 4 0 5 0 7 5 3 4 0 2 1 7,	 fitness - 1, 	czas - 0:00:00.216379
. Q . . Q . . .
. . . . . . . .
. . . . . . . Q
. . . Q . . . .
. . . . . . Q .
Q . . . . . . .
. . Q . . . . .
. . . . . Q . .
Geny: 2 1 6 3 4 7 5 0 7 5 3 4 0 2 1 7,	 fitness - 1, 	czas - 0:00:00.217299
. Q . . . . . .
. . . . Q . . .
. . . . . . . Q
. . . Q . . . .
. . . . . . Q .
Q . . . . . . .
. . Q . . . . .
. . . . . Q . .
Geny: 2 1 6 3 4 6 5 0 7 5 3 4 0 2 1 7,	 fitness - 1, 	czas - 0:00:00.220086
. Q . . Q . . .
. . . . . . . .
. . . . . . . Q
. . . Q . . . .
. . . . . . Q .
Q . . . . . . .
. . Q . . . . .
. . . . . Q . .
Geny: 2 1 6 3 4 7 5 0 7 5 3 4 0 2 1 7,	 fitness - 1, 	cz